# Create an Azure AI Search index and upload documents

This code demonstrate how to use create the index and index documents, that is needed for the notebooks 'find_duplicates.ipynb' and 'rerank_chunks_and_generate_answer.ipynb'.

The output is the index created in the Azure AI Service with documents indexed on it.

## Prerequisites

+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
+ An Azure OpenAI service with the service name and an API key.
+ A deployment of the model on the Azure OpenAI service to calculate the re-ranking.

We used Python 3.12.5, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install openai

## Install packages and create AOAI client

In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import sys
sys.path.append('../..')
from rag_utils import call_aoai

# Load environment variables from .env
load_dotenv(override=True)

aoai_api_version = '2024-02-15-preview'

# AZURE OPENAI FOR RERANKING
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_apikey = os.environ["AZURE_OPENAI_API_KEY"]
aoai_model_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
# Create AOAI client for reranking
aoai_client = AzureOpenAI(
    azure_deployment=aoai_model_name,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint,
    api_key=aoai_apikey
)


## Evaluate the answer comparing with the ground-truth and the question

In [2]:
from rag_utils import evaluate_answer
   

### Test 1: evaluation = 0:
The generated answer and the expected answer have completely different meanings, and the generated answer does not address the user's question.

In [ ]:
# Test 1: evaluation = 0

question = "que promociones de captacion hay?"

expected_answer = """
Actualmente, contamos con dos promociones de captación para nuevas altas en Fusión Digital. La primera es una promoción del 30% de descuento durante 18 meses, que aplica a la totalidad de las cuotas mensuales del servicio, y la segunda es una promoción del 20% de descuento durante 18 meses. Ambas promociones están vigentes desde fechas diferentes, por lo que es importante verificar la fecha de inicio de cada una. Estas promociones solo aplican a clientes que estén dentro del público objetivo y solo para nuevas altas, no para renovaciones ni altas de nuevas sedes con Fusión Digital existentes.
Más información:
""Promociones Fusión Digital (Convergente - PE/ME)"" - (04-06-2023 a 31-12-2036)
"""

answer = """
Actualmente, la promoción de captación disponible es el Contrato Infinito desde el 08/10/2020, que incluye roaming en la Unión Europea, Reino Unido, Islandia, Liechtenstein y Noruega, uso de datos ilimitados en Roaming hasta 50GB y luego a 0,0035€/MB, Movistar Cloud, Conexión Segura y sin contrato de compromiso. El precio promocionado con IVA es de 39,95€/mes y con descuento de cuota mensual es de 29,95€/mes durante 6 meses. La promoción se aplica desde el día de la contratación hasta 184 días después, independientemente del número de facturas que se emitan durante este periodo. En la factura, se puede encontrar el literal 'Promoción Contrato Infinito'.

Otra promoción de captación actual ofrece un 25% extra en líneas móviles durante 4 meses, así como servicios gratuitos como Movistar Cloud, Conexión segura y Nextory. También se puede contratar hasta 5 líneas móviles extra sin ningún tipo de contrato de compromiso. Los precios de las líneas XL y Infinita varían según el plan y la ubicación, y la promoción se aplica desde el día de la contratación hasta 123 días después, independientemente del número de facturas emitidas durante este periodo. La promoción se indica en la factura como 'Promo líneas móviles extra, 4 meses'.
"""

result = evaluate_answer(aoai_client, aoai_model_name, question, expected_answer, answer)
print(f'Evaluation result: {result}')

### Test 2: evaluation = 1:
The generated answer is very similar in meaning to the expected answer but lacks some crucial information, and it partially addresses the user's question.

In [ ]:
# Test 2: evaluation = 1

question = "¿CÓMO DOY DE BAJA UN DISPOSITIVO RENT?"

expected_answer = """
Para dar de baja un dispositivo RENT, se debe utilizar la demanda "Baja dispositivo". Esta demanda se utiliza para desistimientos, baja del dispositivo no asociado a línea y cancelación anticipada del dispositivo. Una vez tramitada la solicitud de baja, el cliente dispone de 15 días para comprar o devolver el dispositivo y se le enviará un SMS. La facturación se "paraliza" unos 20 días a la espera de si el cliente devuelve o no el dispositivo. La baja aparecerá en el parque como una baja de dispositivo.
"""

answer = """
Para dar de baja un dispositivo RENT, el cliente debe seguir el procedimiento de Derecho de Desistimiento (fijo-móvil-satélite) que se encuentra en el punto 2.2 de las condiciones de devolución. El cliente debe informar a la compañía y solicitar la recogida del dispositivo en su domicilio. ((676631)) Una vez que se confirme la recepción del dispositivo en buenas condiciones, el cliente recibirá un SMS y desde Zeleris se gestionará la cancelación del contrato de arrendamiento.
Si el cliente tiene otros dispositivos, como Smartphone, Tablet, consola, smartwatch, portátiles, consolas, barras de sonido, gafas metaquest o aspiradoras, debe seguir el procedimiento de devolución detallado en el punto 2. Para facilitar la gestión, se solicita el código postal del cliente para realizar la petición de la Cita Previa en una tienda cercana. La cita debe ser 72 horas después de la solicitud. También se cuenta con la posibilidad de enviar un SMS al teléfono móvil de contacto del cliente para que él mismo solicite la Cita Previa. 
Si el cliente ya ha devuelto el dispositivo, se confirma que el R2R está dado de baja en parque y se completa el WF. Si el dispositivo no figura dado de baja en parque, se confirma que tiene pedido de recogida en vuelo y se completa el WF. Si la baja no se refleja en Zcontactos, se realiza la operativa CGR 'Baja en parque de Dispositivo R2R ya recogido'. Si el cliente indica que no ha solicitado ni recibido nada, se abre una apertura por CGR por suplantación. Si han pasado más de 14 días y es por error comercial, se envía un correo al buzón de operativas para gestión manual. Cuando se confirma que el equipo esté entregado, se completa el WF.
"""

result = evaluate_answer(aoai_client, aoai_model_name, question, expected_answer, answer)
print(f'Evaluation result: {result}')

### Test 3: evaluation = 2:
The generated answer is well-aligned with the expected answer, capturing the main points accurately, and fully addressing the user's question.

In [ ]:
# Test 3: evaluation = 2

question = "¿CÓMO SE DESISTE DE UN SEGURO DE DISPOSITIVO?"

expected_answer = """
 El seguro puede ser cancelado dentro de los 14 días posteriores a la contratación. Para hacerlo, el cliente puede contactar a la Aseguradora por correo electrónico a produccionequipos@telecommunication.es o por teléfono al 91 606 69 69. El compromiso es de 1 año, renovable por anualidades hasta un máximo de 4 renovaciones, es decir, 5 años de seguro.
"""

answer = """
Para desistir de un seguro de dispositivos, el cliente debe cancelar el seguro en los 14 días posteriores a la contratación. El cliente puede hacerlo contactando con la aseguradora a través de los siguientes medios:
- Correo electrónico: produccion.equipos@telecommunication.es
- Teléfono: 91 606 60 69
Si el cliente decide dar de baja el seguro después de los 14 días, el contrato seguirá en vigor hasta su fecha de vigencia en la cual quedará cancelada, y deberá pagar la totalidad de esa anualidad.
"""

result = evaluate_answer(aoai_client, aoai_model_name, question, expected_answer, answer)
print(f'Evaluation result: {result}')

### Test 4: evaluation = 3:
The generated answer not only aligns with the expected ground truth and answers the user's question but also adds valuable additional details or insights.

In [ ]:
# Test 4: evaluation = 3

question = "¿CÓMO SE DESISTE DE UN SEGURO DE DISPOSITIVO?"

expected_answer = """
 El seguro puede ser cancelado dentro de los 14 días posteriores a la contratación. Para hacerlo, el cliente puede contactar a la Aseguradora por correo electrónico a produccionequipos@telecommunication.es o por teléfono al 91 901 90 69. El compromiso es de 1 año, renovable por anualidades hasta un máximo de 4 renovaciones, es decir, 5 años de seguro.
"""

answer = """
Para desistir de un seguro de dispositivos, el cliente debe cancelar el seguro en los 14 días posteriores a la contratación. El cliente puede hacerlo contactando con la aseguradora a través de los siguientes medios:
- Correo electrónico: produccion.equipos@telecommunication.es
- Teléfono: 91 901 90 69
Si el cliente decide dar de baja el seguro después de los 14 días, el contrato seguirá en vigor hasta su fecha de vigencia en la cual quedará cancelada, y deberá pagar la totalidad de esa anualidad.
"""

result = evaluate_answer(aoai_client, aoai_model_name, question, expected_answer, answer)
print(f'Evaluation result: {result}')